In [90]:
import time
import pymongo
import pprint
from pymongo.errors import WriteError, DuplicateKeyError
from pymongo import MongoClient
from datetime import datetime
import os
client = MongoClient('mongodb://localhost:27017')
db = client.abaco
store2 = db['5']

# Destroy Mongo

In [4]:
import multiprocessing
from multiprocessing import Queue

In [84]:
def threaded_thing(nonce_store):
    print('hey')
    res = nonce_store.update_one(
        {'_id': nonce_key, nonce_id + '.remaining_uses': {'$eq': -1}},
        {'$inc': {nonce_id + '.current_uses': 1},
        '$set': {nonce_id + '.last_use_time': 'hey'}})
    if res.raw_result['updatedExisting'] == True:
        logger.debug("nonce has infinite uses. updating nonce.")
        return 'hey!'

    # Check for remaining uses greater than 0
    res = nonce_store.update_one(
        {'_id': nonce_key, nonce_id + '.remaining_uses': {'$gt': 0}},
        {'$inc': {nonce_id + '.current_uses': 1,
                nonce_id + '.remaining_uses': -1},
        '$set': {nonce_id + '.last_use_time': 'hey'}})

In [6]:
nonce_key = 'DEV-DEVELOP_jane'
nonce_id = 'DEV-DEVELOP_BPbPk0xvz0Llj'
print('hey')
res = nonce_store.update_one(
    {'_id': nonce_key, nonce_id + '.remaining_uses': {'$eq': -1}},
    {'$inc': {nonce_id + '.current_uses': 1},
    '$set': {nonce_id + '.last_use_time': 'hey'}})

hey


In [14]:
nonce_store.update_one(
    {'_id':'did'},
    {'$set':{'stopper':'dopper'}},
    upsert=True)

In [24]:
list(actors_store.find({}))

[{'_id': 'bop', 'hey': 'there'}, {'_id': 'dop', 'hey': 'there'}]

In [23]:
actors_store.update_one(
    {'_id': 'dop'},
    {'$set':{'hey':'there'}},
    upsert=True)

In [11]:
def newBoy(num):
    print(num)
    return(num)

In [ ]:
pool = multiprocessing.Pool(processes=4)
hey = pool.map(newBoy, range(128))
pool.join()
pool.close()

In [176]:
nonce_store.update_one(
    {'_id': 'DEV-DEVELOP_jane'},
    {'$set': {'DEV-DEVELOP_KvVArG21Y1GJ': {'cat':2}}})

## Mongo Store

In [571]:
#### Things to implement:
## getset needs to be fixed for non-maze
## add_if_empty. These two just need to be checked. They work, but errors when you're creating new documents with not yet made fields.

### Store class only:
## set_with_expiry
## insert_one?
## set_with_expiry


### maze
## __setitem__
## __getitem__
## __delitem__
## __len__
## _prepset
## pop_field
## getset
## add_if_empty
## set_with_expiry


###### Need to finish
## insert_one?
## __iter__


class testStore():
    def __init__(self):
        db = client.abaco
        test = db['5']
        self._db = test
        
    def __getitem__(self, key):
        return mongoMaze(self._db, key, [])
        
    def __setitem__(self, key, value):
        self._db.update_one(
            filter={'_id': key},
            update={'$set': {key: value}})
        
    def __delitem__(self, key):
        self._db.delete_one({'_id': key})
    
    def __len__(self):
        return self._db.estimated_document_count()
    
    def __repr__(self):
        return pprint.pformat(list(self._db.find(projection={'_id': False})))
    
    def set_with_expiry(self, key, field, value):
        self._db.update_one(
            filter={'_id': key},
            update={'$set': {'exp': datetime.utcnow(), field: self._prepset(value)}},
            upsert=True)
    
    def _prepset(self, value):
        if type(value) is bytes:
            return value.decode('utf-8')
        return value


# MongoCrawl?
class mongoMaze():
    def __init__(self, db, key, field):
        self._db = db
        self.key = key
        self.field = field
        
        _id = key
        fields -> doc1.subdoc2.subsubdoc3
        
    def __getitem__(self, field):
        self.field.append(field)
        return mongoMaze(self._db, self.key, self.field)
    
    def __setitem__(self, field, value):
        search = self._get_search(field)
        self._db.update_one(
            filter={'_id': self.key},
            update={'$set': {search: value}},
            upsert=True)
        
    def __delitem__(self, field):
        search = self._get_search(field)
        self._db.update_one(
            filter={'_id': self.key},
            update={'$unset': {f'{search}': ''}})
        
    def __len__(self):
        result = self._db.find_one(
            {'_id': self.key},
            projection={'_id': False})
        return len(eval('result' + self._get_index()))
        
    def __repr__(self):
        result = self._db.find_one(
            {'_id': self.key},
            projection={'_id': False})
        return pprint.pformat(eval('result' + self._get_index()))
    
    def pop_field(self, field):
        search = self._get_search(field)
        result = self._db.find_one_and_update(
            filter={'_id': self.key},
            update={'$unset': {f'{search}': ''}})
        return pprint.pformat(eval('result' + self._get_index()))
    
    def getset(self, value):
        search = self._get_search()
        result = self._db.find_one_and_update(
            filter={'_id': self.key},
            update={'$set': {search: value}})
        print(self._get_index())
        return eval('result' + self._get_index())
    
    def add_if_empty(self, value):
        search = self._get_search()
        res = self._db.update_one(
            {'_id': self.key},
            {'$setOnInsert': {search: value}},
            upsert=True)
        print(self.key, search, value, res.raw_result)

        if res.upserted_id:
            return field
        else:
            return None
    
    def _get_search(self, field=None):
        if field:
            self.field.append(field)

        if self.field:
            search = '.'.join(self.field)
        else:
            search = self.key
        return search
            
    def _get_index(self):
        if self.field:
            index = "['" + "']['".join(self.field) + "']"
        else:
            index = ''
        return index
    
    def _prepset(self, value):
        if type(value) is bytes:
            return value.decode('utf-8')
        return value
    
testme = testStore()
#This feels wrong, thus it's discontinued. Also I don't know how to fix it.

In [115]:
testme

{'1': {'2': {'newSubDoc': 'a newer hello'}}}

In [110]:
testme = {'1': {'2': {'3': {'4': 'hello.'}}}}

In [111]:
testme

{'1': {'2': {'3': {'4': 'hello.'}}}}

In [112]:
testme['1']['2']['3']['4'] = hello

'hello.'

In [113]:
testme['1']['2'] = {'newSubDoc': 'a newer hello'}

In [114]:
testme

{'1': {'2': {'newSubDoc': 'a newer hello'}}}

# NEW NEW MONGO 2.0 STORE REDUX DELUXE NEW EDITION

In [625]:
#### Things to implement:
## search

#### Need to check:
## __getitem__ (y, y)
## __setitem__ (y, y)
## __delitem__ (y, y)
## __len__ (y, y)
## __repr__ (y)
## __iter__ - No idea the use case, no modifications done
## _process_inputs (y)
## _prepset (y)
## pop (y, y)
## set_with_expiry (y, y)
## add_if_empty (y, y) - With quirks
## getset - (y, y)

class testStore():
    def __init__(self):
        db = client.abaco
        test = db['5']
        self._db = test
        
    def __getitem__(self, fields):
        """
        Atomically does either:
        Gets and returns 'self[key]' or 'self[key][field1][field2][...]' as a dictionary
        """
        key, _, subscripts = self._process_inputs(fields)
        result = self._db.find_one(
            {'_id': key},
            projection={'_id': False})
        if result == None:
            raise KeyError(f"'_id' of '{key}' not found")
        try:
            return eval('result' + subscripts)
        except KeyError:
            raise KeyError(f"Subscript of {subscripts} does not exists in document of '_id' {key}")

    def __setitem__(self, fields, value):
        """
        Atomically does either:
        Sets 'self[key] = value' or sets 'self[key][field1][field2][...] = value'
        """
        key, dots, _ = self._process_inputs(fields)
        try:
            if isinstance(fields, str) and isinstance(value, dict):
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$set': value},
                    upsert=True)
            else:
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$set': {dots: value}},
                    upsert=True)
        except WriteError:
            raise WriteError(
                "Likely due to trying to set a subfield of a field that does not exists." +
                "\n Try setting a dict rather than a value. Ex. store['id_key', 'key', 'field'] = {'subfield': 'value'}")
        if result.raw_result['nModified'] == 0:
            if not 'upserted' in result.raw_result:
                logger.debug(f'Field not modified, old value likely the same as new. Key: {key}, Fields: {dots}, Value: {value}')

    def __delitem__(self, fields):
        """
        Atomically does either:
        Deletes 'self[key]'
        Unsets 'self[key][field1][field2][...]'
        """
        key, dots, subscripts = self._process_inputs(fields)
        if not subscripts:
            result = self._db.delete_one({'_id': key})
            if result.raw_result['n'] == 0:
                logger.debug(f"No document with '_id' found. Key:{key}, Fields:{dots}")
        else:
            result = self._db.update_one(
                filter={'_id': key},
                update={'$unset': {f'{dots}': ''}})
            if result.raw_result['nModified'] == 0:
                logger.debug(f"Doc with specified fields not found. Key:{key}, Fields:{dots}")

    def __iter__(self):
        for cursor in self._db.find():
            yield cursor['_id']
        # return self._db.scan_iter()

    def __len__(self):
        """
        Returns the estimated document count of a store to give length
        We don't use '.count_documents()' as it's O(N) versus O(1) of estimated
        Length for a document or subdocument comes from len(store['key']['field1'][...]) using dict len()
        """
        return self._db.estimated_document_count()

    def __repr__(self):
        """
        Returns a pretty string of the entire store with '_id' visible for developer use
        """
        return pprint.pformat(list(self._db.find()))

    def _process_inputs(self, fields):
        """
        Takes in fields and returns the key corresponding with '_id', dot notation
        for getting to a specific field in a Mongo query/filter (ex. 'field1.field2.field3.field4')
        and the subscript notation for returning a specified field from a result dictionary
        (ex. `['field1']['field2']['field3']['field4']`)
        """
        if isinstance(fields, str):
            key = dots = fields
            subscripts = ''
        elif isinstance(fields, list) and len(fields) == 1:
            key = dots = fields[0]
            subscripts = ''
        else:
            key = fields[0]
            dots = '.'.join(fields[1:])
            subscripts = "['" + "']['".join(fields[1:]) + "']"
        return key, dots, subscripts

    def _prepset(self, value):
        if type(value) is bytes:
            return value.decode('utf-8')
        return value

    def pop_field(self, fields):
        """
        Atomically pops 'self[key] = value' or 'self[key][field1][field2][...] = value'
        """
        key, dots, subscripts = self._process_inputs(fields)
        if not subscripts:
            result = self._db.find_one(
                {'_id': key},
                projection={'_id': False})
            if result == None:
                raise KeyError(f"'_id' of '{key}' not found")
            del_result = self._db.delete_one({'_id': key})
            if del_result.raw_result['n'] == 0:
                raise KeyError(f"No document deleted")
            return result
        else:
            result = self._db.find_one_and_update(
                filter={'_id': key},
                update={'$unset': {dots: ''}})
            try:
                return eval('result' + subscripts)
            except KeyError:
                raise KeyError(f"Subscript of {subscripts} does not exist in document of '_id' {key}")

    def set_with_expiry(self, fields, value):
        """
        Atomically:
        Sets 'self[key] = value' or 'self[key][field1][field2][...] = value'
        Creates 'exp' subdocument in document root with current time for use with MongoDB TTL expiration index
        Note: MongoDB TTL checks every 60 secs to delete files
        """
        key, dots, _ = self._process_inputs(fields)
        if len(fields) == 1 and isinstance(value, dict):
            result = self._db.update_one(
                filter={'_id': key},
                update={'$set': {'exp': datetime.utcnow()},
                        '$set': value},
                upsert=True)
        else:
            result = self._db.update_one(
                filter={'_id': key},
                update={'$set': {'exp': datetime.utcnow(), dots: self._prepset(value)}},
                upsert=True)

    def full_update(self, key, value):
        result = self._db.update_one(key, value)
        return result

    def getset(self, fields, value):
        """
        Atomically does either:
        Sets 'self[key] = value' and returns previous 'self[key]'
        Sets 'self[key][field1][field2][...] = value' and returns previous 'self[key][field1][field2][...]'
        """
        key, dots, subscripts = self._process_inputs(fields)
        result = self._db.find_one_and_update(
            filter={'_id': key, dots: {'$exists': True}},
            update={'$set': {dots: value}})
        if result == None:
            raise KeyError(f"1Subscript of {subscripts} does not exist in document of '_id' {key}")   
        try:
            if len(fields) == 1:
                return eval(f"result['{key}']")
            else:
                return eval('result' + subscripts)
        except KeyError:
            raise KeyError(f"Subscript of {subscripts} does not exist in document of '_id' {key}")

    def items(self, filter_inp=None, proj_inp={'_id': False}):
        " Either returns all with no inputs, or filters when given filters"
        return list(self._db.find(
            filter=filter_inp,
            projection=proj_inp))

    def add_if_empty(self, fields, value):
        """
        Atomically:
        Sets 'self[key] = value' or 'self[key][field1][field2][...] = value'
        Only if the specified key/field(s) combo does not exist or is empty
        Returns the value if it was added; otherwise, returns None
        Note: Will not override a field set to a value in order to create a subfield
        """
        key, dots, _ = self._process_inputs(fields)
        try:
            if len(fields) == 1 and isinstance(value, dict):
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$setOnInsert': value},
                    upsert=True)
                if result.upserted_id:
                    return key
            elif len(fields) == 1:
                result = self._db.update_one(
                    filter={'_id': key},
                    update={'$setOnInsert': {dots: value}},
                    upsert=True)
                if result.upserted_id:
                    return key
            else:
                try:
                    result = self._db.update_one(
                        filter={'_id': key},
                        update={'$setOnInsert': {dots: value}},
                        upsert=True)
                    if result.upserted_id:
                        return fields
                except WriteError:
                    print("Likely due to trying to set a subfield of a field that is already set to one value")
                    pass
            return None
        except DuplicateKeyError:
            return None
            


actors = testStore()

db = client.abaco

In [384]:
client = MongoClient('mongodb://localhost:27017')
testDB = client.abaco
testSearch = db['6']

In [630]:
for actor in workers.items():
    print(actor)
    break

{'jgJZqGxwX1NRK': {'tenant': 'DEV-DEVELOP', 'id': 'jgJZqGxwX1NRK', 'status': 'READY', 'ch_name': 'worker_jgJZqGxwX1NRK', 'image': 'jstubbs/abaco_test2', 'location': 'unix://var/run/docker.sock', 'cid': 'c54413559b72f737a822e2a4a7ad08374c2d823faf04c76c5c917c2e2ad3d6e4', 'host_id': '0', 'host_ip': '172.17.0.1', 'create_time': '1583872095.400899', 'last_execution_time': 0, 'last_health_check_time': '1583872095.397652'}}


In [633]:
workers.items()

{'jgJZqGxwX1NRK': {'tenant': 'DEV-DEVELOP',
  'id': 'jgJZqGxwX1NRK',
  'status': 'READY',
  'ch_name': 'worker_jgJZqGxwX1NRK',
  'image': 'jstubbs/abaco_test2',
  'location': 'unix://var/run/docker.sock',
  'cid': 'c54413559b72f737a822e2a4a7ad08374c2d823faf04c76c5c917c2e2ad3d6e4',
  'host_id': '0',
  'host_ip': '172.17.0.1',
  'create_time': '1583872095.400899',
  'last_execution_time': 0,
  'last_health_check_time': '1583872095.397652'}}

In [686]:
workers

[{'_id': 'DEV-DEVELOP_7mmLQgoMyvqY3'},
 {'_id': 'DEV-DEVELOP_bvvQQEYgZXXqb'},
 {'_id': 'DEV-DEVELOP_KwwZw7GKlrAZY'},
 {'_id': 'DEV-DEVELOP_7mW7eo4Gp07qa'},
 {'_id': 'DEV-DEVELOP_bvjj0X0eb4oLe'},
 {'_id': 'DEV-DEVELOP_1mRwBqW8MxMR1'},
 {'_id': 'DEV-DEVELOP_jgJKAkeW0xbkQ'},
 {'_id': 'DEV-DEVELOP_KwElbXe7gjyxl'},
 {'_id': 'AGAVE-PROD_jqbR4AYPgeMOG'},
 {'_id': 'AGAVE-PROD_1O05baka5LaLw'},
 {'_id': 'AGAVE-PROD_AJbqxLqOaj3N3'},
 {'_id': 'AGAVE-PROD_4PO5Oqz6xDqmQ'},
 {'_id': 'DEV-STAGING_Bkj4a1x1wK6MG'},
 {'_id': 'DEV-DEVELOP_XYQllNAbDDmJ7'},
 {'_id': 'DEV-DEVELOP_jqWGGArOJ3zZg'},
 {'_id': 'DEV-DEVELOP_bYDL48RK8oZ5J'},
 {'_id': 'DEV-DEVELOP_7PrLJwbmAa0gx'},
 {'_id': 'DEV-DEVELOP_A6x7j3ekWQoX5'},
 {'_id': 'DEV-DEVELOP_8qDlzXLAvA55R'},
 {'_id': 'DEV-DEVELOP_A6DV0mEOrOoP6'},
 {'_id': 'DEV-DEVELOP_7k5R7JWDbyAvk'},
 {'_id': 'DEV-DEVELOP_aka7eyQRx7zoz'},
 {'_id': 'DEV-DEVELOP_KKgkJBx4ozlxr'},
 {'_id': 'DEV-DEVELOP_KKYo0zYLxbPqr'},
 {'_id': 'DEV-DEVELOP_joNoOQMaqaAPx'},
 {'_id': 'DEV-DEVELOP_joNAvvE

In [708]:
amo = 0
actorswnoworkers = 0
for actor in workers.items(proj_inp=None):
    actor_id = actor['_id']
    del actor['_id']
    amo += len(actor)
print(amo)

13


In [683]:
actor_id

'DEV-DEVELOP_7mmLQgoMyvqY3'

In [644]:
(workers.items())

[{'jgJZqGxwX1NRK': {'tenant': 'DEV-DEVELOP',
   'id': 'jgJZqGxwX1NRK',
   'status': 'READY',
   'ch_name': 'worker_jgJZqGxwX1NRK',
   'image': 'jstubbs/abaco_test2',
   'location': 'unix://var/run/docker.sock',
   'cid': 'c54413559b72f737a822e2a4a7ad08374c2d823faf04c76c5c917c2e2ad3d6e4',
   'host_id': '0',
   'host_ip': '172.17.0.1',
   'create_time': '1583872095.400899',
   'last_execution_time': 0,
   'last_health_check_time': '1583873397.518542'}},
 {'b8MyJQObyMNj5': {'tenant': 'DEV-DEVELOP',
   'id': 'b8MyJQObyMNj5',
   'status': 'READY',
   'ch_name': 'worker_b8MyJQObyMNj5',
   'image': 'jstubbs/abaco_test',
   'location': 'unix://var/run/docker.sock',
   'cid': 'b16c853665eedfbcde9cb412868974181ad63180d81a6e0dc7740b68fb3892eb',
   'host_id': '0',
   'host_ip': '172.17.0.1',
   'create_time': '1583872026.253709',
   'last_execution_time': 0,
   'last_health_check_time': '1583873397.896813'}},
 {'b8KbaNBQkq8XA': {'tenant': 'DEV-DEVELOP',
   'id': 'b8KbaNBQkq8XA',
   'status': 'READ

## Search

In [363]:
search = 'COMPLETE'
exact_search = False
if exact_search:
    search = '"\"' + search + '\""'    
tenant = 'DEV-DEVELOP'
user = 'testuser'
skip_amo = 0
limit_amo = 2

In [605]:
# Example request
#'?search=car&exactsearch=DEV-DEVELOP&search=dog&&skip=0&limit=10'

# Mongo aggregation comparison expression operators to implement:
# $eq, $gt, %gte, $lt, $lte, $ne, $in, $nin

def get(search_type, args):
    """
    Placeholder
    """
    #args = dict(args.lists())    
    
    search_type = search_type.lower()
    try:
        search_stores = {'executions': testDB['3'],
                         'workers': testDB['6'],
                         'actors': testDB['5'],
                         'logs': testDB['1']}
        queried_store = search_stores[search_type]
    except KeyError:
        raise KeyError(f'Inputted search_type is invalid, must be one of {list(store_dict.keys())}.')
    
    query, paging = arg_parser(args)
    security, project = provided_dicts(search_type)
    pipeline = query + security + paging #+ project
    
    print('-----PIPELINE-----\n')
    pprint.pprint(pipeline)
    print('\n\n\n-----RESULTANT-----')
    pprint.pprint(list(queried_store.aggregate(pipeline)))
    
def provided_dicts(search_type):
    """
    Placeholder
    """
    security = [{'$match': {'tenant': tenant}},
                {'$lookup':
                    {'from' : '2',
                     'localField' : 'actor_id',
                     'foreignField' : '_id',
                     'as' : 'permissions'}},
                {'$unwind': '$permissions'},
                {'$match': {'permissions.' + curr_user: {'$exists': True}}}]

    project = [{'project': {'ijaiosjdioasjd'}}]
    return security, project

def arg_parser(args):
    """
    Placeholder
    """
    query = []
    search = ""
    skip_amo = None
    limit_amo = None
    
    for key, val in args.items():
        if key == "search":
            if isinstance(val, list):
                joined_val = ' '.join(val)
                search = search + f' {joined_val}'
            else:
                search = search + f' {val}'
        elif key == "exactsearch":
            if isinstance(val, list):
                joined_val = '" "'.join(val)
                search = search + f' "{joined_val}"'
            else:
                search = search + f' "{val}"'
        elif key == "skip":
            if isinstance(val, list):
                raise KeyError('Inputted "skip" paramater is invalid, parameter can only be specified once')
            else:
                skip_amo = int(val)
        elif key == "limit":
            if isinstance(val, list):
                raise KeyError('Inputted "limit" paramater is invalid, parameter can only be specified once')
            else:
                limit_amo = int(val)
        else:
            used_oper = False
            for oper in ['.$eq', '.$gt', '.$gte', '.$lt', '.$lte', '.$ne']:
                if oper in key:
                    key = key.split(oper)[0]
                    oper = oper[1:]
                    query += [{'$match': {key: {oper: val}}}]
                    used_oper = True
                    break
            if not used_oper:
                query += [{'$match': {key: val}}]

    if search:
        search = search[1:] # getting rid of initial space
        query = [{'$match': {'$text': {'$search': search}}},
                 {'$sort': {'score': {'$meta': 'textScore'}}}] + query
    #pprint.pprint(query)
    
    paging = []
    if skip_amo:
        paging = paging + [{'$skip': skip_amo}]
    if limit_amo:
        paging = paging + [{'$limit': limit_amo}]
    #pprint.pprint(paging)

    return query, paging

In [612]:
# '?search=7KEaNE8lrRw5k&search=KUBS&exactsearch=car&exactsearch=POPS&aid=FORDHmongENRY&aid.$gt=100010010'
# args = {'search': ['7KEaNE8lrRw5k', 'KUBS'], 'exactsearch': ['car', 'POPS'], 'aid':'FORDHENRY', 'aid.$gt': '100010010'}

# '?search=DEV-DEVELOP&aid=FORDHENRY&aid.$gt=100010010&skip=0&limit=10'
# args = {'search': 'DEV-DEVELOP', 'aid':'FORDHENRY', 'aid.$gt': '100010010', 'skip':'0', 'limit':'10'}

#args = {'search': 'DEV-DEVELOP'}

#args = {'search': 'DEV-DEVELOP_jJqjmq57Vjv55'}

args = {'exactsearch': 'DEV-DEVELOP_jJqjmq57Vjv55'}

#args = {'search': 'DEV-DEVELOP', 'actor_id': 'DEV-DEVELOP_jJqjmq57Vjv55'}

get('logs', args)

-----PIPELINE-----

[{'$match': {'$text': {'$search': '"DEV-DEVELOP_jJqjmq57Vjv55"'}}},
 {'$sort': {'score': {'$meta': 'textScore'}}},
 {'$match': {'tenant': 'DEV-DEVELOP'}},
 {'$lookup': {'as': 'permissions',
              'foreignField': '_id',
              'from': '2',
              'localField': 'actor_id'}},
 {'$unwind': '$permissions'},
 {'$match': {'permissions.testuser': {'$exists': True}}}]



-----RESULTANT-----
[{'_id': 'jJEjDGD1bXOAQ',
  'actor_id': 'DEV-DEVELOP_jJqjmq57Vjv55',
  'exp': datetime.datetime(2020, 3, 9, 17, 16, 47, 849000),
  'logs': "Contents of MSG: {'status_message': ' ', 'tenant_id': "
          "'DEV-DEVELOP', 'event_type': 'ACTOR_READY', 'event_time_utc': "
          "'1583774204.154498', 'event_time_display': '2020-03-09 "
          "17:16:44.154498', 'actor_id': '8wVKOXGxRklDq', 'actor_dbid': "
          "'DEV-DEVELOP_8wVKOXGxRklDq', '_abaco_link': '', '_abaco_webhook': "
          "'http://nginx/actors/jJqjmq57Vjv55/messages?x-nonce=DEV-DEVELOP_bjY4yK

## Example of database indexing

In [712]:
# Do this to create a wildcard index on littttterally everything.
testDB['1'].create_index([('$**', pymongo.TEXT)])

'$**_text'

## Initial basic pipeline for logs database

In [713]:
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}},
            {'$match': {'tenant': tenant}},
            {'$lookup':
                {'from' : '2',
                 'localField' : 'actor_id',
                 'foreignField' : '_id',
                 'as' : 'permissions'}},
            {'$unwind': '$permissions'},
            {'$match': {'permissions.' + curr_user: {'$exists': True}}},
            {'$skip': skip_amo},
            {'$limit': limit_amo}]
            #{'project': {ijaiosjdioasjd}}

list(testDB['1'].aggregate(pipeline))

[]

## Conditional aggregation

In [717]:
#ex. Update logs fields if actors exists
pipeline = [{'$match': {'$text': {'$search': search}}},
            {'$sort': {'score': {'$meta': 'textScore'}}},
            {'$match': {'tenant': tenant}},
            {'$lookup':
                {'from' : '2',
                 'localField' : 'actor_id',
                 'foreignField' : '_ifd',
                 'as' : 'permissions'}},
            {'$unwind': '$permissions'},
            {'$match': {'permissions.' + curr_user: {'$exists': True}}},
            {'$skip': skip_amo},
            {'$limit': limit_amo}]
            #{'project': {ijaiosjdioasjd}}

db = '2' 
    
pipeline = [{'$lookup':
                {'from' : '5', #Actors_db
                 'localField' : '_id',
                 'foreignField' : '_id',
                 'as' : 'actor_dict'}},
            {'$unwind': '$actor_dict'},
           ]

list(testDB['3'].aggregate(pipeline))

[{'_id': 'DEV-DEVELOP_BRWEDexDp3qXR',
  'BmRRBZ7k0jD63': {'tenant': 'DEV-DEVELOP',
   'api_server': 'https://dev.tenants.develop.tacc.cloud',
   'actor_id': 'DEV-DEVELOP_BRWEDexDp3qXR',
   'executor': 'testuser',
   'worker_id': '1vlBVlLQBlMbm',
   'id': 'BmRRBZ7k0jD63',
   'message_received_time': '1584021699.283508',
   'start_time': '1584021699.796293',
   'runtime': 3,
   'cpu': 102726568,
   'io': 1142,
   'status': 'COMPLETE',
   'exit_code': 0,
   'final_state': {'Status': 'exited',
    'Running': False,
    'Paused': False,
    'Restarting': False,
    'OOMKilled': False,
    'Dead': False,
    'Pid': 0,
    'ExitCode': 0,
    'Error': '',
    'StartedAt': '2020-03-12T14:01:40.1653518Z',
    'FinishedAt': '2020-03-12T14:01:42.1744322Z'}},
  'j0MJrM75bW3AG': {'tenant': 'DEV-DEVELOP',
   'api_server': 'https://dev.tenants.develop.tacc.cloud',
   'actor_id': 'DEV-DEVELOP_BRWEDexDp3qXR',
   'executor': 'testuser',
   'worker_id': '1vlBVlLQBlMbm',
   'id': 'j0MJrM75bW3AG',
   'messa

## Vague search w/ sorting by score

In [ ]:
list(testDB['1']
    .find({'$text':{'$search': "event_type"}},
          {'score':{'$meta': "textScore"}})
    .sort([('score', {'$meta': "textScore"})]))

In [334]:
list(testSearch.list_indexes())

[SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'abaco.6')])]